# 국토교통부_연립다세대 전월세 실거래자료_크롤링


- 데이터 출처: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15058038   
- 지역코드 출처: 주민등록주소코드 변경내역(2018. 1. 22. 현재) https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=61552 내 첨부파일 jscode20180122.zip   
   
##### *주의: 일일 트래픽 1000회 제한*

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

## 함수 설정

In [4]:
def collect_RH_trade(ym, lawd_cd) :
    
    # 제공받은 API 
    API_KEY = 'bHg%2FpafJkgCqoGEFC3cI5IvynqkFuwK53BYhBu3bqMz7I%2BX%2BEYH8R9jV%2FCMc06dm11DO%2FksvWzs5sIUeyCjiTQ%3D%3D'
    
    # URL
    url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHTrade'
 
    # 요청메시지
    url = url + '?&LAWD_CD=' + lawd_cd + '&DEAL_YMD=' + ym + '&serviceKey=' + API_KEY
 
    resultXML = urlopen(url)
    result = resultXML.read()
    xmlsoup = BeautifulSoup(result, 'lxml-xml')
 
    te = xmlsoup.findAll('item')
 
    # 데이터프레임 결과물
    RH_trade = pd.DataFrame()
 
    for t in te:
        deal_amt = t.find('거래금액').text
        try:
            build_y = t.find("건축년도").text
        except:
            build_y = ""
        deal_y = t.find('년').text
        deal_m = t.find('월').text
        deal_day = t.find('일').text
        area_land = t.find('대지권면적').text
        RH_nm = t.find('연립다세대').text
        dong = t.find('법정동').text
        area_exc = t.find('전용면적').text
        region_code = t.find('지역코드').text

        try:
            region_num = t.find("지번").text
        except:
            region_num = ""
 
        floor = t.find('층').text
 
        temp = pd.DataFrame(([[deal_amt, build_y, deal_y, deal_m, deal_day, area_land, RH_nm, dong, area_exc, region_code, region_num, floor]]),
                            columns = ['deal_amt', 'build_y', 'deal_y', 'deal_m', 'deal_day', 'area_land', 'RH_nm', 'dong', 'area_exc', 'region_code', 'region_num', 'floor'])
        RH_trade = pd.concat([RH_trade, temp])
 
    RH_trade = RH_trade.reset_index(drop=True)
 
    return RH_trade

## 수집

#### 수집 경과 ####
2020년 data => 7/16 완료    
2019년 data => 7/16 트래픽 초과 => 2019년 2월    

### **** 2019년 2월 경기도 데이터 이후 수집 예정 ****

In [5]:
if __name__ == "__main__":
    # 지역코드 가져오기
    code = pd.read_excel("KIKcd_B.20180122.xlsx")
    code_cap = code[(code["시도명"]=="서울특별시") | (code["시도명"] == "경기도")]
    code_cap = code_cap[code_cap["읍면동명"].isnull() == True]
    code_cap = code_cap[code_cap["시군구명"].isnull() == False]
    code_cap["region_code"] = code_cap["법정동코드"].astype(str).str[0:5]
    code_cap = code_cap.reset_index(drop=True)
    RH_trade = pd.DataFrame()

    # 기간설정(연월: 'yyyymm')
    ym = list(['201902'])
 
    for m in ym:
        i = 0
        for co in code_cap['region_code'] :
            temp = collect_RH_trade(m, co)
            temp['sido'] = code_cap['시도명'][i]
            temp['sigungu'] = code_cap['시군구명'][i]
            RH_trade = pd.concat([RH_trade, temp])
            i += 1
            print(co + ", " + str(len(temp)) + "개 완료")
            
        print("******** " + str(m) + " 완료 ********")

11110, 41개 완료
11140, 10개 완료
11170, 44개 완료
11200, 22개 완료
11215, 64개 완료
11230, 56개 완료
11260, 56개 완료
11290, 118개 완료
11305, 95개 완료
11320, 89개 완료
11350, 40개 완료
11380, 223개 완료
11410, 82개 완료
11440, 131개 완료
11470, 188개 완료
11500, 192개 완료
11530, 105개 완료
11545, 60개 완료
11560, 65개 완료
11590, 88개 완료
11620, 100개 완료
11650, 68개 완료
11680, 51개 완료
11710, 101개 완료
11740, 76개 완료
41110, 0개 완료
41111, 48개 완료
41113, 106개 완료
41115, 57개 완료
41117, 12개 완료
41130, 0개 완료
41131, 41개 완료
41133, 124개 완료
41135, 18개 완료
41150, 81개 완료
41170, 0개 완료
41171, 61개 완료
41173, 33개 완료
41190, 436개 완료
41210, 119개 완료
41220, 69개 완료
41250, 45개 완료
41270, 0개 완료
41271, 130개 완료
41273, 115개 완료
41280, 0개 완료
41281, 82개 완료
41285, 27개 완료
41287, 12개 완료
41290, 4개 완료
41310, 31개 완료
41360, 97개 완료
41370, 18개 완료
41390, 85개 완료
41410, 21개 완료
41430, 25개 완료
41450, 11개 완료
41460, 0개 완료
41461, 69개 완료
41463, 20개 완료
41465, 13개 완료
41480, 61개 완료
41500, 29개 완료
41550, 6개 완료
41570, 65개 완료
41590, 81개 완료
41610, 240개 완료
41630, 19개 완료
41650, 38개 완료
41670, 19개 완료
41800, 7개 완료


In [6]:
RH_trade_2019 = pd.read_csv('RH_trade_2019.csv')

In [7]:
RH_trade_2019

,deal_amt,build_y,deal_y,deal_m,deal_day,area_land,RH_nm,dong,area_exc,region_code,region_num,floor,sido,sigungu
0,"87,000",2000,2019,1,9,64.3517,백악관,창성동,121.90,11110,90,4,서울특별시,종로구
1,"12,000",2000,2019,1,18,14.1500,청마F동,누상동,34.65,11110,74,1,서울특별시,종로구
2,"23,000",1996,2019,1,30,14.5400,상하그린맨션4,누상동,38.77,11110,24,1,서울특별시,종로구
3,"22,800",2001,2019,1,30,13.7950,인왕4차넥스빌,누상동,42.82,11110,166-104,4,서울특별시,종로구
4,"50,500",2004,2019,1,23,41.2700,예지명가,옥인동,79.25,11110,142,3,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9279,"27,000",2018,2019,2,28,33.3600,제이파크 빌,퇴계원읍 퇴계원리,50.94,41360,97-20,2,경기도,남양주시
9280,"19,000",2017,2019,2,28,44.2570,베스트빌,퇴계원읍 퇴계원리,43.88,41360,288-37,3,경기도,남양주시
9281,"5,300",2002,2019,2,11,43.7920,한솔빌리지(B동),수동면 내방리,39.58,41360,342-2,4,경기도,남양주시
9282,"10,750",2012,2019,2,16,66.4000,"노블레스(가,나동)",수동면 입석리,79.98,41360,335-10,2,경기도,남양주시


In [8]:
RH_trade_2019.drop(RH_trade_2019.loc[(RH_trade_2019['sido']=='경기도') & (RH_trade_2019['deal_m'] == 2)].index, inplace=True)

In [9]:
RH_trade_2019

,deal_amt,build_y,deal_y,deal_m,deal_day,area_land,RH_nm,dong,area_exc,region_code,region_num,floor,sido,sigungu
0,"87,000",2000,2019,1,9,64.3517,백악관,창성동,121.90,11110,90,4,서울특별시,종로구
1,"12,000",2000,2019,1,18,14.1500,청마F동,누상동,34.65,11110,74,1,서울특별시,종로구
2,"23,000",1996,2019,1,30,14.5400,상하그린맨션4,누상동,38.77,11110,24,1,서울특별시,종로구
3,"22,800",2001,2019,1,30,13.7950,인왕4차넥스빌,누상동,42.82,11110,166-104,4,서울특별시,종로구
4,"50,500",2004,2019,1,23,41.2700,예지명가,옥인동,79.25,11110,142,3,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7531,"26,500",2018,2019,2,22,16.0300,광나루에스엠파크,천호동,29.30,11740,332-19,4,서울특별시,강동구
7532,"27,630",2018,2019,2,23,15.6400,청운빌리지(574),천호동,25.98,11740,574,6,서울특별시,강동구
7533,"21,500",2003,2019,2,27,29.5910,(143-8),천호동,50.79,11740,143-8,3,서울특별시,강동구
7534,"23,000",2000,2019,2,27,26.9200,천호하이츠,천호동,59.05,11740,363-31,1,서울특별시,강동구


In [10]:
RH_trade_201902gg = RH_trade

In [12]:
RH_trade_2019 = pd.concat([RH_trade_2019, RH_trade_201902gg])

In [13]:
if __name__ == "__main__":
    # 지역코드 가져오기
    code = pd.read_excel("KIKcd_B.20180122.xlsx")
    code_cap = code[(code["시도명"]=="서울특별시") | (code["시도명"] == "경기도")]
    code_cap = code_cap[code_cap["읍면동명"].isnull() == True]
    code_cap = code_cap[code_cap["시군구명"].isnull() == False]
    code_cap["region_code"] = code_cap["법정동코드"].astype(str).str[0:5]
    code_cap = code_cap.reset_index(drop=True)
    RH_trade = pd.DataFrame()

    # 기간설정(연월: 'yyyymm')
    ym = list(['201903', '201904', '201905', '201906', 
               '201907', '201908', '201909', '201910', '201911', '201912'])
 
    for m in ym:
        i = 0
        for co in code_cap['region_code'] :
            temp = collect_RH_trade(m, co)
            temp['sido'] = code_cap['시도명'][i]
            temp['sigungu'] = code_cap['시군구명'][i]
            RH_trade = pd.concat([RH_trade, temp])
            i += 1
            print(co + ", " + str(len(temp)) + "개 완료")
            
        print("******** " + str(m) + " 완료 ********")

11110, 41개 완료
11140, 17개 완료
11170, 85개 완료
11200, 24개 완료
11215, 134개 완료
11230, 64개 완료
11260, 128개 완료
11290, 112개 완료
11305, 116개 완료
11320, 97개 완료
11350, 74개 완료
11380, 305개 완료
11410, 127개 완료
11440, 147개 완료
11470, 138개 완료
11500, 283개 완료
11530, 240개 완료
11545, 130개 완료
11560, 67개 완료
11590, 94개 완료
11620, 150개 완료
11650, 77개 완료
11680, 31개 완료
11710, 129개 완료
11740, 138개 완료
41110, 0개 완료
41111, 71개 완료
41113, 141개 완료
41115, 138개 완료
41117, 35개 완료
41130, 0개 완료
41131, 63개 완료
41133, 150개 완료
41135, 20개 완료
41150, 91개 완료
41170, 0개 완료
41171, 80개 완료
41173, 57개 완료
41190, 541개 완료
41210, 113개 완료
41220, 71개 완료
41250, 30개 완료
41270, 0개 완료
41271, 195개 완료
41273, 119개 완료
41280, 0개 완료
41281, 121개 완료
41285, 32개 완료
41287, 13개 완료
41290, 3개 완료
41310, 40개 완료
41360, 144개 완료
41370, 16개 완료
41390, 91개 완료
41410, 35개 완료
41430, 29개 완료
41450, 7개 완료
41460, 0개 완료
41461, 101개 완료
41463, 26개 완료
41465, 20개 완료
41480, 69개 완료
41500, 101개 완료
41550, 9개 완료
41570, 149개 완료
41590, 120개 완료
41610, 292개 완료
41630, 36개 완료
41650, 67개 완료
41670, 38개 완료
4

41290, 9개 완료
41310, 51개 완료
41360, 287개 완료
41370, 21개 완료
41390, 72개 완료
41410, 99개 완료
41430, 155개 완료
41450, 13개 완료
41460, 0개 완료
41461, 109개 완료
41463, 37개 완료
41465, 15개 완료
41480, 96개 완료
41500, 52개 완료
41550, 20개 완료
41570, 66개 완료
41590, 60개 완료
41610, 285개 완료
41630, 31개 완료
41650, 38개 완료
41670, 52개 완료
41800, 6개 완료
41820, 16개 완료
41830, 28개 완료
******** 201910 완료 ********
11110, 77개 완료
11140, 27개 완료
11170, 176개 완료
11200, 59개 완료
11215, 198개 완료
11230, 99개 완료
11260, 173개 완료
11290, 282개 완료
11305, 341개 완료
11320, 133개 완료
11350, 147개 완료
11380, 446개 완료
11410, 135개 완료
11440, 255개 완료
11470, 280개 완료
11500, 456개 완료
11530, 171개 완료
11545, 156개 완료
11560, 87개 완료
11590, 186개 완료
11620, 253개 완료
11650, 185개 완료
11680, 136개 완료
11710, 287개 완료
11740, 173개 완료
41110, 0개 완료
41111, 184개 완료
41113, 209개 완료
41115, 138개 완료
41117, 29개 완료
41130, 0개 완료
41131, 92개 완료
41133, 227개 완료
41135, 61개 완료
41150, 179개 완료
41170, 0개 완료
41171, 215개 완료
41173, 252개 완료
41190, 670개 완료
41210, 403개 완료
41220, 66개 완료
41250, 23개 완료
41270, 0개 완료
41271, 2

In [14]:
RH_trade.tail()

,deal_amt,build_y,deal_y,deal_m,deal_day,area_land,RH_nm,dong,area_exc,region_code,region_num,floor,sido,sigungu
19,"18,000",2019,2019,12,5,78.19,노브힐빌라,옥천면 신복리,67.73,41830,1106-1,1,경기도,양평군
20,"21,500",2017,2019,12,14,28.2781,골든타워,용문면 다문리,56.5552,41830,785-4,9,경기도,양평군
21,"20,800",2018,2019,12,14,29.0201,개성하이빌,용문면 다문리,59.9,41830,792-4,8,경기도,양평군
22,"19,000",2019,2019,12,29,53.5043,리치빌,용문면 다문리,72.8,41830,389-13,4,경기도,양평군
23,"16,000",2011,2019,12,20,84.4375,용문청재하이빌(A동),용문면 조현리,72.795,41830,64,3,경기도,양평군


In [16]:
RH_trade_2019 = pd.concat([RH_trade_2019, RH_trade])

In [17]:
RH_trade_2019.to_csv('RH_trade_2019_new.csv', index=False)